# Blocking and opening paths in a DAG

This is a fake-data simulation demonstrating the effect of adjusting for certain variables in a multiple regression.

In [ ]:
import pymc3 as pm
import numpy as np
import arviz as az
from scipy import stats

In [ ]:
coeff_AU = 1
coeff_AC = -2
coeff_CB = -2
coeff_UB = 3
coeff_UX = 1
coeff_CY = 1
coeff_XY = 1.5

alpha_U = 0
alpha_B = 0
alpha_C = 0
alpha_X = 0
alpha_Y = 0

A = stats.norm.rvs(0, 1, size = 100)
U = stats.norm.rvs(alpha_U + coeff_AU * A, 0.1)
C = stats.norm.rvs(alpha_C + coeff_AC * A, 0.1)
B = stats.norm.rvs(alpha_B + coeff_CB * C + coeff_UB * U, 0.1)
X = stats.norm.rvs(alpha_X + coeff_UX * U, 0.1)
Y = stats.norm.rvs(alpha_Y + coeff_CY * C + coeff_XY * X, 0.1)

In [ ]:
with pm.Model() as model_x:
    alpha = pm.Normal('alpha', 0, 1)
    beta_x = pm.Normal('beta_x', 0, 1)
    sigma = pm.HalfCauchy('sigma', 1)
    
    mu = alpha + beta_x * X
    
    y_ = pm.Normal('y', mu, sigma, observed = Y)
    trace_x = pm.sample()
    summary_x_only = az.summary(trace_x)

In [ ]:
summary_x_only

In [ ]:
with pm.Model() as model_xa:
    alpha = pm.Normal('alpha', 0, 1)
    beta_x = pm.Normal('beta_x', 0, 1)
    beta_a = pm.Normal('beta_a', 0, 1)
    sigma = pm.HalfCauchy('sigma', 1)
    
    
    mu = alpha + beta_x * X + beta_a * A
    
    y_ = pm.Normal('y', mu, sigma, observed = Y)
    trace_xa = pm.sample(target_accept = 0.9)
    summary_xa = az.summary(trace_xa)

In [22]:
summary_xa

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,-0.022,0.014,-0.049,0.004,0.000,0.000,1280.0,1028.0,1.0
beta_x,1.455,0.092,1.292,1.623,0.003,0.002,840.0,803.0,1.0
beta_a,-1.950,0.094,-2.124,-1.786,0.003,0.002,836.0,804.0,1.0
sigma,0.144,0.011,0.124,0.163,0.000,0.000,1133.0,1058.0,1.0


In [ ]:
with pm.Model() as model_xc:
    alpha = pm.Normal('alpha', 0, 1)
    beta_x = pm.Normal('beta_x', 0, 1)
    beta_c = pm.Normal('beta_c', 0, 1)
    sigma = pm.HalfCauchy('sigma', 1)
    
    mu = alpha + beta_x * X + beta_c * C
    
    y_ = pm.Normal('y', mu, sigma, observed = Y)
    trace_xc = pm.sample()
    summary_xc = az.summary(trace_xc)

In [ ]:
summary_xc

In [ ]:
with pm.Model() as model_xb:
    alpha = pm.Normal('alpha', 0, 1)
    beta_x = pm.Normal('beta_x', 0, 1)
    beta_b = pm.Normal('beta_b', 0, 1)
    sigma = pm.HalfCauchy('sigma', 1)
    
    mu = alpha + beta_x * X + beta_b * B 
    
    y_ = pm.Normal('y', mu, sigma, observed = Y)
    trace_xb = pm.sample()
    summary_xb = az.summary(trace_xb)

In [ ]:
summary_xb

In [ ]:
with pm.Model() as model_xab:
    alpha = pm.Normal('alpha', 0, 1)
    beta_x = pm.Normal('beta_x', 0, 1)
    beta_a = pm.Normal('beta_a', 0, 1)
    beta_b = pm.Normal('beta_b', 0, 1)
    sigma = pm.HalfCauchy('sigma', 1)
    
    mu = alpha + beta_x * X + beta_b * B + beta_a * A 
    y_ = pm.Normal('y', mu, sigma, observed = Y)
    trace_xab = pm.sample(target_accept = 0.9)
    summary_xab = az.summary(trace_xab)

In [ ]:
summary_xab

In [ ]:
with pm.Model() as model_all:
    alpha = pm.Normal('alpha', 0, 1)
    beta_x = pm.Normal('beta_x', 0, 1)
    beta_a = pm.Normal('beta_a', 0, 1)
    beta_b = pm.Normal('beta_b', 0, 1)
    beta_c = pm.Normal('beta_c', 0, 1)
    sigma = pm.HalfCauchy('sigma', 1)
    
    mu = alpha + beta_x * X + beta_b * B + beta_a * A + beta_c * C
    y_ = pm.Normal('y', mu, sigma, observed = Y)
    trace_all = pm.sample(target_accept = 0.9)
    summary_all = az.summary(trace_all)

In [ ]:
with pm.Model() as model_xac:
    alpha = pm.Normal('alpha', 0, 1)
    beta_x = pm.Normal('beta_x', 0, 1)
    beta_a = pm.Normal('beta_a', 0, 1)
    beta_c = pm.Normal('beta_c', 0, 1)
    sigma = pm.HalfCauchy('sigma', 1)
    
    mu = alpha + beta_x * X + beta_a * A + beta_c * C
    y_ = pm.Normal('y', mu, sigma, observed = Y)
    trace_xac = pm.sample(target_accept = 0.9)
    summary_xac = az.summary(trace_xac)

In [ ]:
summary_xac = az.summary(trace_xac)
summary_xac

In [ ]:
summary_all

In [21]:
az.compare({'x':trace_x, 'xa':trace_xa, 'xc':trace_xc, 'xab':trace_xab, 'xb':trace_xb, 'xac':trace_xac, 'all':trace_all}, ic='loo', scale='log')

/home/dylan/.conda/envs/info510/lib/python3.8/site-packages/arviz/stats/stats.py:145: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  warnings.warn(
/home/dylan/.conda/envs/info510/lib/python3.8/site-packages/arviz/data/io_pymc3.py:96: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(
/home/dylan/.conda/envs/info510/lib/python3.8/site-packages/arviz/data/io_pymc3.py:96: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(
/home/dylan/.conda/envs/info510/lib/python3.8/site-packages/arviz/data/io_pymc3.py:9

,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
xc,0,79.053928,4.222298,0.000000,8.931187e-01,7.045579,0.000000,False,log
xac,1,78.257148,5.308144,0.796780,0.000000e+00,7.006836,1.003234,False,log
all,2,77.037978,6.468685,2.015950,0.000000e+00,7.079371,0.984532,False,log
xab,3,61.395911,5.065132,17.658017,5.923398e-02,6.609152,6.931699,False,log
xb,4,54.298409,4.322094,24.755519,4.764732e-02,7.187053,8.194708,False,log
xa,5,50.234070,4.115162,28.819857,0.000000e+00,7.282522,7.670840,False,log
x,6,-33.437035,2.983270,112.490963,2.387828e-10,6.390677,8.944861,False,log


## Parents / Grandparents example

In [ ]:
b_GC = 0
b_PC = 1
b_U = 1